# Generating Sequential Data with PrivBayes

# Environment

## Library Imports

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import sys
from itertools import product
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from copy import deepcopy
import math
from dython.nominal import associations
import matplotlib.pyplot as plt
import random

In [3]:
print(module_path)

C:\Users\mwo2008.54063\Documents\GitHub\thesis


## Jupyter-specific Imports and Settings

In [4]:
# set printing options
np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    get_ipython().run_line_magic('load_ext', 'autoreload')

get_ipython().run_line_magic('autoreload', '2')
from importlib import reload

## Import local libraries

In [5]:
from synthesis.synthesizers.privbayes import PrivBayes, NodeParentPair

## Load original data

In [6]:
tumours = pd.read_pickle('../../../../../Master Thesis/data/preprocessed/tumours_patients_2017_train.pickle').set_index('eid')
treatments = pd.read_pickle('../../../../../Master Thesis/data/preprocessed/treatments_2017_train.pickle')

In [7]:
tumours.shape, treatments.shape

((191913, 8), (317592, 4))

#### Transform treatments to main group

In [8]:
th_gebeurtenis = pd.read_csv('../../../../../Master Thesis/data/thesauri/th_gebeurtenis.csv', engine='python')

In [9]:
# Add stoma main group
def get_stoma_maingroup(row):
    if 'stoma' in row['omschrijving'].lower():
        return 'STOMA'
    else:
        return row['hoofdgroep_code']
    
th_gebeurtenis['hoofdgroep_code'] = th_gebeurtenis.apply(lambda row: get_stoma_maingroup(row), axis=1)

In [10]:
maingroup_dict = dict(zip(th_gebeurtenis['code'], th_gebeurtenis['hoofdgroep_code']))

In [11]:
treatments['gbs_gebeurtenis_code'] = treatments['gbs_gebeurtenis_code'].replace(maingroup_dict)

## Sequential Data Pivoting + Pre- and Post-processing DP

In [12]:
class SequentialDataPivotingDP:
    """
    Basic class for Sequential Data Pivoting as described in Section 4.2.2, extended with DP approach (Section 4.4.2):
    * Appends sequence length attribute to patient covariates
    * Post-processing synthetic data by 
        (1) cutting the treatment sequence once the first 'empty' treatment is sampled by 
            the generative model (i.e., treating the next treatments as 'empty' regardless of the value sampled), and 
        (2) For those synthetic patients with all treatments sampled (i.e., no ’empty’ value sampled in the treatment 
            attributes),the treatment sequence is cut at the synthetic value for the ’Sequence length’ attribute for this 
            patient
    """
    
    def __init__(self, max_n_treatments, primary_key, treatment_col, order_col):
        self.max_n_treatments = max_n_treatments
        self.primary_key = primary_key
        self.treatment_col = treatment_col
        self.order_col = order_col
        
    def transform(self, df_static, df_treatments):
        """
        Flattens df_treatments and appends to df_static, extensions:
        * Appends sequence length attribute to patient covariates
        """
        # Add number of treatments column
        sequence_lengths = pd.DataFrame(df_treatments.groupby(self.primary_key)[self.treatment_col].count()).rename(
                        {self.treatment_col: 'sequence_length'}, axis=1)
        
        df_static = df_static.merge(sequence_lengths, left_index=True, right_index=True)
        
        # Sort values by order col
        df_treatments = df_treatments.sort_values([self.primary_key, self.order_col])
        
        # Group treatments by primary key, list treatments
        df_treatments_grouped = pd.DataFrame(df_treatments.groupby(self.primary_key)[self.treatment_col].apply(list))
        
        # Instantiate flat treatments dataframe
        df_treatments_flat = pd.DataFrame(index=df_treatments_grouped.index)
        
        # Flatten treatment columns
        for nr in range(1, self.max_n_treatments + 1):
            df_treatments_flat['treatment_' + str(nr)] = df_treatments_grouped[self.treatment_col].apply(self.get_treatment_nr,
                                                                                                        args=[nr])
            
        # Merge flat treatments dataframe with static covariates dataframe
        df = df_static.merge(df_treatments_flat, left_index=True, right_index=True)
        
        return df
    
    def inverse_transform(self, df):
        """
        Transforms flat dataframe back to df_static and df_treatments, extensions:
        * Post-processing synthetic data by 
            (1) cutting the treatment sequence once the first 'empty' treatment is sampled by 
                the generative model (i.e., treating the next treatments as 'empty' regardless of the value sampled), and 
            (2) For those synthetic patients with all treatments sampled (i.e., no ’empty’ value sampled in the treatment 
                attributes),the treatment sequence is cut at the synthetic value for the ’Sequence length’ attribute for this 
                patient
        """
        # Define static and treatment columns
        static_cols = [col for col in df if not (col.startswith('treatment') or col=='sequence_length')]
        treatment_cols = list(set(df.columns) - set(static_cols))
        
        # df_static
        df_static = df[static_cols].copy()
        
        # df_treatments
        # Set all columns after sequence_length to nan if all treatment columns are sampled
        def fix_nr_treatments(row):
            if not 'nan' in list(row.values)[-self.max_n_treatments:]:
                nr_treatments = int(row['sequence_length'])
            
                for nr in range(nr_treatments+1, self.max_n_treatments+1):
                    treatment_col = 'treatment_' + str(nr)
                    row[treatment_col] = 'nan'

            return row
        
        df = df.copy()
        df = df.apply(lambda row: fix_nr_treatments(row), axis=1)
        
        # Treatment columns
        treatment_cols_treatments = sorted([col for col in treatment_cols if col.startswith('treatment')])
        
        # One df per treatment column to reverse flatten
        treatment_cols_data = [pd.DataFrame(df[col]) for col in treatment_cols_treatments]
        for a_df in treatment_cols_data:
            a_df['nr_treatment'] = a_df.columns[0][-1]
            a_df.rename(columns={a_df.columns[0]: self.treatment_col}, inplace=True)
        
        # Concat all treatment columns to one dataframe
        treatments = pd.concat(treatment_cols_data)
        treatments.index.name = self.primary_key
        treatments = treatments.reset_index()
        
        # Replace nan 
        df_treatments = treatments.replace('nan', np.nan)
        df_treatments = df_treatments.dropna().rename({'nr_treatment': self.order_col}, axis=1)
        
        # Sort by primary key and order of treatments
        df_treatments = df_treatments.sort_values([self.primary_key, self.order_col]).reset_index(drop=True)

        # Drop all treatments after first nan
        df_treatments = df_treatments[df_treatments[self.order_col].astype(int)
                                      ==df_treatments.groupby(self.primary_key).cumcount()+1].copy()
                
        return df_static, df_treatments
        
    @staticmethod
    def get_treatment_nr(treatments, nr):
        if len(treatments)>=nr:
            return treatments[nr-1]
        else:
            return np.nan

In [13]:
# develop transformer to flatten the table
fts = SequentialDataPivotingDP(max_n_treatments=5,
                               primary_key='eid',
                               treatment_col='gbs_gebeurtenis_code',
                               order_col='gbs_vnr')

df = fts.transform(tumours, treatments)

In [14]:
df.shape

(191913, 14)

In [15]:
# Set all dtypes to string
df = df.astype(str)

# Generate data

### Epsilon 1

In [29]:
epsilon = 1

#### Static data only to find static network

In [30]:
df_static = df.iloc[:, :9]

In [33]:
pb = PrivBayes(epsilon=epsilon, verbose=True)
pb.fit(df_static)

1/9 - Root of network: sequence_length

2/9 - Evaluating next node to add to network
Number of NodeParentPair candidates: 8
Candidates: [NodeParentPair(node='tum_topo_sublokalisatie_code', parents=('sequence_length',)), NodeParentPair(node='stadium', parents=('sequence_length',)), NodeParentPair(node='pat_geslacht_code', parents=('sequence_length',)), NodeParentPair(node='tum_differentiatiegraad_code', parents=('sequence_length',)), NodeParentPair(node='tum_topo_code', parents=('sequence_length',)), NodeParentPair(node='age_at_diagnosis', parents=('sequence_length',)), NodeParentPair(node='survival_1', parents=('sequence_length',)), NodeParentPair(node='tum_lymfklieren_positief_atl', parents=('sequence_length',))]
Selected node: 'tum_lymfklieren_positief_atl' - with parents: ('sequence_length',)

3/9 - Evaluating next node to add to network
Number of NodeParentPair candidates: 7
Candidates: [NodeParentPair(node='tum_topo_sublokalisatie_code', parents=('sequence_length', 'tum_lymfkliere

Selected node: 'tum_topo_code' - with parents: ('tum_topo_sublokalisatie_code', 'sequence_length', 'stadium')

8/9 - Evaluating next node to add to network
Number of NodeParentPair candidates: 37
Candidates: [NodeParentPair(node='age_at_diagnosis', parents=('survival_1', 'sequence_length', 'pat_geslacht_code', 'tum_topo_sublokalisatie_code')), NodeParentPair(node='age_at_diagnosis', parents=('tum_topo_code', 'tum_topo_sublokalisatie_code', 'survival_1', 'pat_geslacht_code')), NodeParentPair(node='age_at_diagnosis', parents=('sequence_length', 'tum_topo_code', 'tum_topo_sublokalisatie_code', 'survival_1')), NodeParentPair(node='age_at_diagnosis', parents=('tum_topo_code', 'tum_topo_sublokalisatie_code', 'sequence_length', 'pat_geslacht_code')), NodeParentPair(node='age_at_diagnosis', parents=('tum_topo_sublokalisatie_code', 'survival_1', 'pat_geslacht_code', 'stadium')), NodeParentPair(node='age_at_diagnosis', parents=('tum_topo_code', 'tum_topo_sublokalisatie_code', 'stadium')), NodePa

Selected node: 'age_at_diagnosis' - with parents: ('tum_topo_code', 'tum_topo_sublokalisatie_code', 'sequence_length', 'pat_geslacht_code')

Learned Network Structure

Learning conditional probabilities: sequence_length - with parents None ~ estimated size: 5
Learning conditional probabilities: tum_lymfklieren_positief_atl - with parents ('sequence_length',) ~ estimated size: 15
Learning conditional probabilities: pat_geslacht_code - with parents ('sequence_length', 'tum_lymfklieren_positief_atl') ~ estimated size: 30
Learning conditional probabilities: tum_topo_sublokalisatie_code - with parents ('sequence_length', 'pat_geslacht_code', 'tum_lymfklieren_positief_atl') ~ estimated size: 300
Learning conditional probabilities: stadium - with parents ('tum_topo_sublokalisatie_code', 'sequence_length', 'tum_lymfklieren_positief_atl') ~ estimated size: 1050
Learning conditional probabilities: survival_1 - with parents ('tum_topo_sublokalisatie_code', 'sequence_length', 'pat_geslacht_code', 

In [34]:
network_static = deepcopy(pb.network_)

In [35]:
network_static

[NodeParentPair(node='sequence_length', parents=None),
 NodeParentPair(node='tum_lymfklieren_positief_atl', parents=('sequence_length',)),
 NodeParentPair(node='pat_geslacht_code', parents=('sequence_length', 'tum_lymfklieren_positief_atl')),
 NodeParentPair(node='tum_topo_sublokalisatie_code', parents=('sequence_length', 'pat_geslacht_code', 'tum_lymfklieren_positief_atl')),
 NodeParentPair(node='stadium', parents=('tum_topo_sublokalisatie_code', 'sequence_length', 'tum_lymfklieren_positief_atl')),
 NodeParentPair(node='survival_1', parents=('tum_topo_sublokalisatie_code', 'sequence_length', 'pat_geslacht_code', 'stadium')),
 NodeParentPair(node='tum_topo_code', parents=('tum_topo_sublokalisatie_code', 'sequence_length', 'stadium')),
 NodeParentPair(node='tum_differentiatiegraad_code', parents=('stadium', 'sequence_length', 'survival_1', 'tum_lymfklieren_positief_atl')),
 NodeParentPair(node='age_at_diagnosis', parents=('tum_topo_code', 'tum_topo_sublokalisatie_code', 'sequence_length

#### Sequential network definition

In [38]:
context_columns = ['tum_topo_code', 'stadium', 'survival_1']
max_treatments = 5

init_network_sequential = []
for treatment_nr in range(1, max_treatments+1):
    if not treatment_nr==1:
        init_network_sequential += ([NodeParentPair('treatment_' + str(treatment_nr), 
                                                       context_columns + ['treatment_' + str(treatment_nr-1)])])
    else:
        init_network_sequential += [NodeParentPair('treatment_' + str(treatment_nr), context_columns + ['age_at_diagnosis'])]

In [39]:
init_network_sequential

[NodeParentPair(node='treatment_1', parents=['tum_topo_code', 'stadium', 'survival_1', 'age_at_diagnosis']),
 NodeParentPair(node='treatment_2', parents=['tum_topo_code', 'stadium', 'survival_1', 'treatment_1']),
 NodeParentPair(node='treatment_3', parents=['tum_topo_code', 'stadium', 'survival_1', 'treatment_2']),
 NodeParentPair(node='treatment_4', parents=['tum_topo_code', 'stadium', 'survival_1', 'treatment_3']),
 NodeParentPair(node='treatment_5', parents=['tum_topo_code', 'stadium', 'survival_1', 'treatment_4'])]

#### Add sequential network to static network

In [40]:
init_network = network_static + init_network_sequential

#### Run PrivBayes with initialized network

In [41]:
# Train
pb = PrivBayes(epsilon=epsilon, verbose=True)
pb.set_network(init_network)
pb.fit(df)

1/14 - init node sequence_length - with parents: None
2/14 - init node tum_lymfklieren_positief_atl - with parents: ('sequence_length',)
3/14 - init node pat_geslacht_code - with parents: ('sequence_length', 'tum_lymfklieren_positief_atl')
4/14 - init node tum_topo_sublokalisatie_code - with parents: ('sequence_length', 'pat_geslacht_code', 'tum_lymfklieren_positief_atl')
5/14 - init node stadium - with parents: ('tum_topo_sublokalisatie_code', 'sequence_length', 'tum_lymfklieren_positief_atl')
6/14 - init node survival_1 - with parents: ('tum_topo_sublokalisatie_code', 'sequence_length', 'pat_geslacht_code', 'stadium')
7/14 - init node tum_topo_code - with parents: ('tum_topo_sublokalisatie_code', 'sequence_length', 'stadium')
8/14 - init node tum_differentiatiegraad_code - with parents: ('stadium', 'sequence_length', 'survival_1', 'tum_lymfklieren_positief_atl')
9/14 - init node age_at_diagnosis - with parents: ('tum_topo_code', 'tum_topo_sublokalisatie_code', 'sequence_length', 'pat

In [42]:
# Synthesize data
df_pb = pb.sample()

Number of records generated: 191913 / 191913
Synthetic Data Generated



#### Post-processing

In [43]:
df_static, _ = fts.inverse_transform(df)
df_treatments = treatments.sort_values(['eid', 'gbs_vnr'])
df_static_pb, df_treatments_pb = fts.inverse_transform(df_pb)

In [44]:
df_static_pb.to_pickle('../synthetic_data/PBS_eps1_tumours.pickle')
df_treatments_pb.to_pickle('../synthetic_data/PBS_eps1_treatments.pickle')

### Epsilon 0.1

In [60]:
epsilon = 0.1

#### Static data only to find static network

In [61]:
df_static = df.iloc[:, :9]

In [64]:
pb = PrivBayes(epsilon=epsilon, verbose=True)
pb.fit(df_static)

1/9 - Root of network: stadium

2/9 - Evaluating next node to add to network
Number of NodeParentPair candidates: 8
Candidates: [NodeParentPair(node='tum_topo_sublokalisatie_code', parents=('stadium',)), NodeParentPair(node='sequence_length', parents=('stadium',)), NodeParentPair(node='pat_geslacht_code', parents=('stadium',)), NodeParentPair(node='tum_differentiatiegraad_code', parents=('stadium',)), NodeParentPair(node='tum_topo_code', parents=('stadium',)), NodeParentPair(node='age_at_diagnosis', parents=('stadium',)), NodeParentPair(node='survival_1', parents=('stadium',)), NodeParentPair(node='tum_lymfklieren_positief_atl', parents=('stadium',))]
Selected node: 'survival_1' - with parents: ('stadium',)

3/9 - Evaluating next node to add to network
Number of NodeParentPair candidates: 7
Candidates: [NodeParentPair(node='tum_topo_sublokalisatie_code', parents=('survival_1', 'stadium')), NodeParentPair(node='sequence_length', parents=('survival_1', 'stadium')), NodeParentPair(node='p

Selected node: 'tum_differentiatiegraad_code' - with parents: ('tum_topo_code', 'stadium')

8/9 - Evaluating next node to add to network
Number of NodeParentPair candidates: 30
Candidates: [NodeParentPair(node='sequence_length', parents=('age_at_diagnosis', 'survival_1', 'pat_geslacht_code')), NodeParentPair(node='sequence_length', parents=('tum_topo_sublokalisatie_code', 'survival_1')), NodeParentPair(node='sequence_length', parents=('tum_topo_sublokalisatie_code', 'pat_geslacht_code')), NodeParentPair(node='sequence_length', parents=('age_at_diagnosis', 'tum_differentiatiegraad_code')), NodeParentPair(node='sequence_length', parents=('survival_1', 'pat_geslacht_code', 'tum_differentiatiegraad_code')), NodeParentPair(node='sequence_length', parents=('tum_topo_code', 'age_at_diagnosis', 'pat_geslacht_code')), NodeParentPair(node='sequence_length', parents=('tum_topo_code', 'pat_geslacht_code', 'tum_differentiatiegraad_code')), NodeParentPair(node='sequence_length', parents=('tum_topo_c

In [65]:
network_static = deepcopy(pb.network_)

In [66]:
network_static

[NodeParentPair(node='stadium', parents=None),
 NodeParentPair(node='survival_1', parents=('stadium',)),
 NodeParentPair(node='pat_geslacht_code', parents=('survival_1', 'stadium')),
 NodeParentPair(node='age_at_diagnosis', parents=('survival_1', 'pat_geslacht_code', 'stadium')),
 NodeParentPair(node='tum_topo_sublokalisatie_code', parents=('age_at_diagnosis', 'pat_geslacht_code')),
 NodeParentPair(node='tum_topo_code', parents=('age_at_diagnosis', 'tum_topo_sublokalisatie_code')),
 NodeParentPair(node='tum_differentiatiegraad_code', parents=('tum_topo_code', 'stadium')),
 NodeParentPair(node='tum_lymfklieren_positief_atl', parents=('survival_1', 'pat_geslacht_code', 'stadium')),
 NodeParentPair(node='sequence_length', parents=('age_at_diagnosis', 'pat_geslacht_code', 'tum_lymfklieren_positief_atl'))]

#### Sequential network definition

In [67]:
context_columns = ['tum_topo_code', 'stadium', 'survival_1']
max_treatments = 5

init_network_sequential = []
for treatment_nr in range(1, max_treatments+1):
    if not treatment_nr==1:
        init_network_sequential += ([NodeParentPair('treatment_' + str(treatment_nr), 
                                                       context_columns + ['treatment_' + str(treatment_nr-1)])])
    else:
        init_network_sequential += [NodeParentPair('treatment_' + str(treatment_nr), context_columns + ['age_at_diagnosis'])]

In [68]:
init_network_sequential

[NodeParentPair(node='treatment_1', parents=['tum_topo_code', 'stadium', 'survival_1', 'age_at_diagnosis']),
 NodeParentPair(node='treatment_2', parents=['tum_topo_code', 'stadium', 'survival_1', 'treatment_1']),
 NodeParentPair(node='treatment_3', parents=['tum_topo_code', 'stadium', 'survival_1', 'treatment_2']),
 NodeParentPair(node='treatment_4', parents=['tum_topo_code', 'stadium', 'survival_1', 'treatment_3']),
 NodeParentPair(node='treatment_5', parents=['tum_topo_code', 'stadium', 'survival_1', 'treatment_4'])]

#### Add sequential network to static network

In [69]:
init_network = network_static + init_network_sequential

#### Run PrivBayes with initialized network

In [70]:
# Train
pb = PrivBayes(epsilon=epsilon, verbose=True)
pb.set_network(init_network)
pb.fit(df)

1/14 - init node stadium - with parents: None
2/14 - init node survival_1 - with parents: ('stadium',)
3/14 - init node pat_geslacht_code - with parents: ('survival_1', 'stadium')
4/14 - init node age_at_diagnosis - with parents: ('survival_1', 'pat_geslacht_code', 'stadium')
5/14 - init node tum_topo_sublokalisatie_code - with parents: ('age_at_diagnosis', 'pat_geslacht_code')
6/14 - init node tum_topo_code - with parents: ('age_at_diagnosis', 'tum_topo_sublokalisatie_code')
7/14 - init node tum_differentiatiegraad_code - with parents: ('tum_topo_code', 'stadium')
8/14 - init node tum_lymfklieren_positief_atl - with parents: ('survival_1', 'pat_geslacht_code', 'stadium')
9/14 - init node sequence_length - with parents: ('age_at_diagnosis', 'pat_geslacht_code', 'tum_lymfklieren_positief_atl')
10/14 - init node treatment_1 - with parents: ['tum_topo_code', 'stadium', 'survival_1', 'age_at_diagnosis']
11/14 - init node treatment_2 - with parents: ['tum_topo_code', 'stadium', 'survival_1'

In [71]:
# Synthesize data
df_pb = pb.sample()

Number of records generated: 191913 / 191913
Synthetic Data Generated



#### Post-processing

In [72]:
df_static, _ = fts.inverse_transform(df)
df_treatments = treatments.sort_values(['eid', 'gbs_vnr'])
df_static_pb, df_treatments_pb = fts.inverse_transform(df_pb)

In [73]:
df_static_pb.to_pickle('../synthetic_data/PBS_eps0.1_tumours.pickle')
df_treatments_pb.to_pickle('../synthetic_data/PBS_eps0.1_treatments.pickle')